## Trabalho de Conclusão de Curso - TCC
#### Turma - DSA T212
#### Aluno : Drausio Gomes dos Santos
#### Orientadora: Gabrielle Maria Romeiro Lombardi
#### Tema: Construção e uso de modelos de grafos para representação e análise da estrutura organizacional da administração pública federal.
### Resultados do projeto

## Etapa 2: Pré processamento dos dados
<p> Procedimentos para recuperação e tratamento das fontes de dados originais, de forma a serem utilizadas em linguagem de programação que possibilite a construção de modelo de grafos</p>

##### Parâmetros iniciais do caderno notebook

In [1]:
base_path = 'C:/projetos/tcc_dsa_usp_esalq/dados/'
#base_path = '/content/drive/My Drive/Colab Notebooks/'
url_base_dou = "http://dados.gov.br/dados/conjuntos-dados/diario-oficial-da-uniao-materias-publicadas-em-"
url_base_siorg = "http://estruturaorganizacional.dados.gov.br/doc/"

#### Importando as bibliotecas


In [2]:
!pip install wget
!pip install selenium

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [3]:
!python --version
import os
from os import listdir
from os.path import isfile, join
from io import StringIO
from zipfile import ZipFile
from xml.etree import ElementTree
import time
from datetime import datetime

print("")
import selenium
print("selenium: " + selenium.__version__+ " - (https://pypi.org/project/selenium/)") 

import pandas as pd
print("pandas: " + pd.__version__+ " - (https://pypi.org/project/pandas/)") 

import wget   
print("wget: " +wget.__version__+ " - (https://pypi.org/project/wget/)")

import urllib.request as req
print("urllib.request: " +req.__version__+ " - (https://docs.python.org/3/library/urllib.request.html)")

import bs4
from bs4 import BeautifulSoup
print("bs4: " + bs4.__version__+ " - (https://pypi.org/project/bs4/)")

import requests
print("requests: " + requests.__version__ + " - (https://pypi.org/project/requests/)")

import csv
print("csv: " + csv.__version__ + " - (https://pypi.org/project/csv/)")

import pytz
print("pytz: " + pytz.__version__  + " - (https://pypi.org/project/pytz/)")




Python 3.9.12

selenium: 4.7.2 - (https://pypi.org/project/selenium/)
pandas: 1.4.2 - (https://pypi.org/project/pandas/)
wget: 3.2 - (https://pypi.org/project/wget/)
urllib.request: 3.9 - (https://docs.python.org/3/library/urllib.request.html)
bs4: 4.11.1 - (https://pypi.org/project/bs4/)
requests: 2.27.1 - (https://pypi.org/project/requests/)
csv: 1.0 - (https://pypi.org/project/csv/)
pytz: 2021.3 - (https://pypi.org/project/pytz/)


#### Funções globais

In [4]:
class Main:     
    def show_current_time(self):
        ini = time.time()
        tz_SP = pytz.timezone('America/Sao_Paulo') 
        now = datetime.now(tz_SP)
        current_time = now.strftime("%d/%m/%Y %H:%M:%S")
        print("Current Time =", current_time)  

main = Main()

#### Montando a estrutura de diretórios para os dados do DOU

In [5]:
def monta_pastas_secoes(base_path, month):
    os.chdir(base_path)
    dir_secoes = ['01','02','03']
    for secao in dir_secoes:
        isdir = os.path.isdir(base_path+'/'+secao)  
        if not isdir:
            print("criando diretorio do mes: " + month + " seção: " + secao)
            os.mkdir(base_path +'/'+secao) 
    
def monta_pastas_meses(bash_path, year):
    os.chdir(bash_path)
    months = ['01','02','03','04','05','06','07','08','09','10','11','12']
    for month in months:
        isdir = os.path.isdir(bash_path+"/"+month)
        if not isdir:
            print("criando diretorio do mes " + month +"/"+str(year))
            os.mkdir(bash_path +"/"+month)
        monta_pastas_secoes(bash_path+"/"+month, month)

def monta_pastas_anos(base_path):
    path=base_path+"dou"
    os.chdir(path)
    years = range(2002,datetime.today().year+1)
    for year in years:
        isdir = os.path.isdir(path+"/"+str(year))  
        if not isdir:
            print("criando diretorio do ano " + str(year))
            os.mkdir(path+"/"+str(year))    
        monta_pastas_meses(path+"/"+str(year),year)
    
def montando_estrutura_de_pastas():
    monta_pastas_anos(base_path)
    
def criando_diretorio_raiz():
    isdir = os.path.isdir(base_path) 
    if not isdir:
        print("criando diretório raiz em : "+ base_path)
        os.mkdir(base_path)
    else:
        print("diretório raiz "+ base_path + "    já existente")    
    os.chdir(base_path)
    dirname = "dou"
    isdir = os.path.isdir(base_path+dirname) 
    if not isdir:
        print("criando diretório dou em : "+ base_path+dirname)
        os.mkdir(dirname)
    else:
        print("diretório dou  "+ base_path+dirname + " já existente")
    
print("Início da criação da árvore de diretórios.")    
main.show_current_time()

criando_diretorio_raiz()
montando_estrutura_de_pastas()
               
print("Término da criação da árvore de diretórios.")
main.show_current_time()


Início da criação da árvore de diretórios.
Current Time = 11/01/2023 10:03:11
diretório raiz C:/projetos/tcc_dsa_usp_esalq/dados/    já existente
diretório dou  C:/projetos/tcc_dsa_usp_esalq/dados/dou já existente
Término da criação da árvore de diretórios.
Current Time = 11/01/2023 10:03:12


#### Carregamento dos arquivos .zip com xml da base DOU do portal de dados abertos do governo federal ( nova rotina em fase de desenvolvimento em virtude de mudanças no portal de dados abertos ) .
###### Obs. Enquanto se dá a adequação do código os arquivos devem ser baixados manualmente e colocados na estrutura de diretório criada nos passos anteriores

In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

def executeTest():
    global driver
    driver.get('https://dados.gov.br/dados/conjuntos-dados/diario-oficial-da-uniao-materias-publicadas-em-2022')
    time.sleep(7)
    element = driver.find_elements(By.XPATH, '//header/button[./span[text()="Recursos"]]')
    element[0].click()
    time.sleep(3)
    element1 = driver.find_elements(By.XPATH, '//div/button[text()=" Acessar o recurso "]')
    #print(element1[0].tag_name)
    element1[0].click()
    time.sleep(7)

def startWebDriver():
    global driver
    options = Options()
    options.add_argument("--disable-infobars")
    driver = webdriver.Chrome(options=options)


startWebDriver()
executeTest()
driver.quit()

In [7]:
def baixa_arquivos(parsed_html):
    for link in parsed_html.body.findAll('a', attrs={'class':'resource-url-analytics'}):
        url = link.get('href').split("?url=")[1]
        filename = "S"+link.get('href').split('/S')[-1][0:8]
        mes = filename[-6:-4]
        ano = filename[-4:]
        secao = filename[-8:-6]
        path = base_path+"dou/"+year+'/'+mes+'/'+secao+'/'
        file_download = path + filename + '.zip'
        isfile = os.path.isfile(file_download) 
        os.chdir(path) 
        if mes in meses :
            #print(file_download)
            #print(url)
            if not isfile:
                try:    
                    wget.download(url)   
                    print(' SUCESSO = download zip file to: ' + file_download + ' from url: '+url)
                except:
                    print(' ERRO = downloading zip file to: ' + file_download + ' from url: '+url)
            else:
                print(' arquivo já carregado: ' + file_download)

#anos = ['2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015',
#         '2016','2017','2018','2019','2020','2021','2022']

anos=['2019','2020','2021','2022']
meses=['01','02','03','04','05','06','07','08','09','10','11','12']


#print("Início do download dos arquivos zip do atos do DOU")
main.show_current_time()
print("--------------------------------------------------")

for year in anos:
    urlopen =  url_base_dou + year 
    if year == '2017':
        urlopen = 'http://dados.gov.br/dataset/diario-oficial-da-uniao'
    #resp = req.urlopen(urlopen)
    #conteudo = resp.read(int(resp.getheader('Content-Length')))  
    #parsed_html = BeautifulSoup(StringIO(conteudo.decode("UTF-8")))
    #baixa_arquivos(parsed_html)
    #print("Concluído o download dos arquivos zip do ano :" + year)

print("--------------------------------------------------")
#print("Término do download dos arquivos zip do atos do DOU")
print("Rotina desativada")
main.show_current_time()



Current Time = 11/01/2023 10:03:35
--------------------------------------------------
--------------------------------------------------
Rotina desativada
Current Time = 11/01/2023 10:03:35


#### Gerando arquivos .CSV a partir do XML de artigos do DOU.
<p> Arquivo XML origem </p>

```xml
<?xml version="1.0"?>
<xml>
<article idMateria="19841008" highlightimagename="" highlightimage="" highlight="" highlightPriority="" highlightType="" editionNumber="164-B" pdfPage="http://pesquisa.in.gov.br/imprensa/jsp/visualiza/index.jsp?data=29/08/2022&jornal=601&pagina=1" numberPage="1" artNotes="" artSize="12" artCategory="Ministério da Economia/Gabinete do Ministro" artClass="00015:00001:00000:00000:00000:00000:00000:00000:00000:00000:00024:00001" pubDate="29/08/2022" artType="Portaria Conjunta" pubName="DO1E" idOficio="9058105" name="Portaria Conjunta ME.BCB 7679 - " id="27738641">
<body>
<Identifica>
<![CDATA[  PORTARIA CONJUNTA ME/BCB Nº 7.679, DE 25 DE AGOSTO DE 2022]]>
</Identifica>
<Data>
<![CDATA[]]>
</Data>
<Ementa>
<![CDATA[ Dispõe sobre a rentabilidade dos títulos da Dívida Pública Mobiliária Federal interna em poder do Banco Central do Brasil, a remuneração das disponibilidades de caixa da União e o percentual assegurado às entidades públicas que aplicam na Conta Única do Tesouro Nacional.]]>
</Ementa>
<Titulo/>
<SubTitulo/>
<Texto>
<![CDATA[<p class="identifica">PORTARIA CONJUNTA ME/BCB Nº 7.679, DE 25 DE AGOSTO DE 2022</p><p class="ementa">Dispõe sobre a rentabilidade dos títulos da Dívida Pública Mobiliária Federal interna em poder do Banco Central do Brasil, a remuneração das disponibilidades de caixa da União e o percentual assegurado às entidades públicas que aplicam na Conta Única do Tesouro Nacional.</p><p>O MINISTRO DE ESTADO DA ECONOMIA, no uso da atribuição que lhe confere o art. 87, parágrafo único, inciso II, da Constituição, e o PRESIDENTE DO BANCO CENTRAL DO BRASIL, no uso de suas atribuições, tendo em vista o disposto na Medida Provisória nº 2.179-36, de 24 de agosto de 2001, resolvem:</p><p>Art. 1º Esta Portaria Conjunta dispõe sobre a rentabilidade dos títulos da Dívida Pública Mobiliária Federal interna em poder do Banco Central do Brasil, a remuneração das disponibilidades de caixa da União e o percentual assegurado às entidades públicas que aplicam na Conta Única do Tesouro Nacional.</p><p>Art. 2º A rentabilidade intrínseca dos títulos da Dívida Pública Mobiliária Federal interna de responsabilidade do Tesouro Nacional em poder do Banco Central do Brasil, de que trata o art. 1º da Medida Provisória nº 2.179-36, de 24 de agosto de 2001, compreende a taxa interna de retorno, calculada a partir do preço médio unitário de aquisição dos respectivos títulos pelo Banco Central do Brasil, acrescida da atualização do valor nominal de cada título, quando houver.</p><p>Parágrafo único. Para fins de cálculo diário da taxa interna de retorno dos títulos de que trata o caput, deverão ser utilizados duzentos e cinquenta e dois dias úteis como referência para o período de um ano.</p><p>Art. 3º Os títulos da Dívida Pública Mobiliária Federal interna de responsabilidade do Tesouro Nacional objeto de operações compromissadas de venda com compromisso de recompra, realizadas pelo Banco Central do Brasil com o mercado, integram a carteira com base na qual é calculada a taxa média aritmética ponderada das rentabilidades intrínsecas de que trata o art. 1º da Medida Provisória nº 2.179-36, de 2001.</p><p>Art. 4º O Banco Central do Brasil informará à Secretaria do Tesouro Nacional da Secretaria Especial do Tesouro e Orçamento do Ministério da Economia, no prazo de até dois dias úteis, o fator diário de remuneração das disponibilidades de caixa da União, bem como colocará à disposição, por meio eletrônico, dentre outras informações consideradas relevantes:</p><p>I - as memórias de cálculo que deram origem ao valor informado; e</p><p>II - o quadro demonstrativo da taxa interna de retorno utilizada no cálculo da rentabilidade intrínseca de cada título.</p><p>Parágrafo único. O fator diário de remuneração das disponibilidades de caixa da União é calculado com base na média aritmética ponderada da rentabilidade intrínseca dos títulos da Dívida Pública Mobiliária Federal interna de emissão do Tesouro Nacional em poder do Banco Central do Brasil.</p><p>Art. 5º Será assegurada, às entidades de que trata o § 2º do art. 2º da Medida Provisória nº 2.170-36, de 23 de agosto de 2001, a remuneração equivalente a noventa e oito por cento da remuneração paga pelo Banco Central do Brasil sobre os saldos da conta única do Tesouro Nacional.</p><p>Art. 6º Fica revogada a Portaria nº 6.224, de 13 de julho de 2022, do Ministério da Economia.</p><p>Parágrafo único. Ficam convalidados os atos praticados com base na Portaria nº 6.224, de 2022, do Ministério da Economia.</p><p>Art. 7º Esta Portaria entra em vigor na data de sua publicação.</p><p class="assina">PAULO GUEDES</p><p class="cargo">Ministro de Estado da Economia</p><p class="assina">ROBERTO DE OLIVEIRA CAMPOS NETO</p><p class="cargo">Presidente do Banco Central do Brasil</p>]]>
</Texto>
</body>
<Midias/>
</article>
</xml>
```

<p> Arquivo CSV destino </p>

| campo      | descrição |tipo| exemplo|
| ----------- | ----------- |||
| id | Identificador do artigo. Obtido a partir do nome do arquivo xml sem a extensão | String | 601_20220829_19841008|
| tipo| Tipo do artigo do DOU (Portaria, Portaria Conjunta, Ato Declaratório, Ato, Aviso, etc...) | string | Ato Declaratório |
| emissor| Unidade organizacional emissora do ato precedido pela hierárquia | String |Ministério da Economia/Gabinete do Ministro|
| data| Data da publicação no DOU no formato DD/MM/AAAA | Date | 29/08/2022 |
| responsavel| Responsável pela assinatura do ato | String | ROBERTO DE OLIVEIRA CAMPOS NETO|
| cargo| Cargo do responsável pelo ato | String | Presidente do Banco Central do Brasil |



In [8]:
def monta_linha(artigo,id,txt):
    tipo= artigo.attrib['artType']
    data=artigo.attrib['pubDate']
    emissor=artigo.attrib['artCategory']
    responsavel=""
    cargo=""
    if txt is not None:
        texto = txt.split("\"assina\">")
        if(len(texto)>1):
            texto1 = texto[1].split("</p><p class=\"cargo\">")  
            responsavel = texto1[0].split("<")[0]
            if "cargo" in texto[1] and len(texto1) > 1:
                cargo = texto1[1].split("</p>")[0]
            responsavel=responsavel
            
    return [id,tipo,emissor,data,responsavel,cargo] 

def parser_xml_file(myfile,csvwriterunico):
    id=myfile.name[:-4]  
    tree = ElementTree.parse(myfile)  
    artigo = tree.getroot().find('article')
    texto = artigo.find('body').find('Texto')
    linha = monta_linha(artigo,id,texto.text)
    #contador = contador + 1
    csvwriterunico.writerow(linha) 
    #count_ok = count_ok + 1
    #except:
    #    print("Error on parsing ..." + myfile.name)
    #    count_error = count_error + 1
    

def le_arquivo_zip(file_download,csvwriterunico):
    isfile = os.path.isfile(file_download) 
    #print(file_download)
    if isfile:  
        with ZipFile(file_download, 'r') as myzip:
            for name in myzip.namelist():
                with myzip.open(name) as myfile:                            
                    if myfile.name[-3:]  == 'xml':
                        parser_xml_file(myfile,csvwriterunico)                        
            myzip.close()

def carrega_arquivo_csv(contador,count_error,csvwriterunico):
    for mes in meses:
        inim = time.time()
        print(mes + "|", end = '')
        for secao in secoes:
            inip = time.time()
            path = base_path +"dou/" + ano+'/'+mes+'/'+secao+'/'
            os.chdir(path)      
            count_ok = 0
            mypath = path
            file_download = path + 'S' + secao + mes + ano + '.zip'
            le_arquivo_zip(file_download,csvwriterunico)            
    

anos = ['2019','2020','2021','2022']
meses = ['01','02','03','04','05','06',\
         '07','08','09','10','11','12']
secoes= ['01','02','03']          


print("Início da geração dos arquivos .csv para cada ano")
main.show_current_time()
print("--------------------------------------------------")

for ano in anos:
    ini = time.time()
    contador = 0
    count_error = 0  
    pasta = base_path +"dou/" + ano +'/'
    os.chdir(pasta) 
    isfile = os.path.isfile(ano+'.csv') 
    #print(file_download)
    if not isfile: 
        csvfileunico = open(ano+'.csv','w', newline='', encoding='utf-8')
        csvwriterunico=csv.writer(csvfileunico)
        col_names = ['id','tipo','emissor','data','responsavel','cargo'] 
        csvwriterunico.writerow(col_names)
        print(pasta + ano+'.csv')    
        carrega_arquivo_csv(contador,count_error,csvwriterunico)
        csvfileunico.close()
    else:
        print("arquivo já carregado: " + pasta + ano+'.csv')
    fim = time.time()

    print("Tempo total: " + ano + ": " + str(fim-ini))
    main.show_current_time()
    #print("qtd OK" , contador)
    #print("qtd ERRO" , count_error)
    
print("--------------------------------------------------")
print("Término do download dos arquivos zip do atos do DOU")
main.show_current_time()


Início da geração dos arquivos .csv para cada ano
Current Time = 11/01/2023 10:03:35
--------------------------------------------------
arquivo já carregado: C:/projetos/tcc_dsa_usp_esalq/dados/dou/2019/2019.csv
Tempo total: 2019: 0.001001119613647461
Current Time = 11/01/2023 10:03:35
arquivo já carregado: C:/projetos/tcc_dsa_usp_esalq/dados/dou/2020/2020.csv
Tempo total: 2020: 0.0
Current Time = 11/01/2023 10:03:35
arquivo já carregado: C:/projetos/tcc_dsa_usp_esalq/dados/dou/2021/2021.csv
Tempo total: 2021: 0.0
Current Time = 11/01/2023 10:03:35
arquivo já carregado: C:/projetos/tcc_dsa_usp_esalq/dados/dou/2022/2022.csv
Tempo total: 2022: 0.0
Current Time = 11/01/2023 10:03:35
--------------------------------------------------
Término do download dos arquivos zip do atos do DOU
Current Time = 11/01/2023 10:03:35
